In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import time
all_matches=[]
years=[2023,2022]
all_matches=[]
standing_url="https://fbref.com/en/comps/9/Premier-League-Stats"
for y in years:
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text)
    standing_table = soup.select("table.stats_table")[0]
   
    links = standing_table.find_all("a")
    links = [ l.get("href") for l in links]
    links = [l for l in links if "/squads/" in l]
    links = [f"https://fbref.com{l}" for l in links]
    previous_season = soup.select("a.prev")[0].get("href")
    standing_url=f"https://fbref.com{previous_season}"
    for l in links:
        team_n = l.split("/")[-1].replace("-Stats","").replace("-"," ")
        data = requests.get(l)
        matches= pd.read_html(StringIO(data.text),match = "Scores & Fixtures")[0]
        soup= BeautifulSoup(data.text)
        nlinks = soup.find_all("a")
        
        nlinks = [i.get("href") for i in nlinks]
        nlinks = [i for i in nlinks if i and "all_comps/shooting/" in i]
      
        
        data = requests.get(f"https://fbref.com{nlinks[0]}")
        
        shooting = pd.read_html(StringIO(data.text),match ="Shooting")[0]
        shooting.columns=shooting.columns.droplevel()

        try: 
             team = matches.merge(shooting[["Date","Sh","SoT","Dist", "FK", "PK", "PKatt"]], on = "Date")
        except ValueError:
            continue
        team = team[team["Comp"] == "Premier League"]
        team["Seasons"]=y
        team["Team"]=team_n
        all_matches.append(team)
        time.sleep(2)
    
match_tot = pd.concat(all_matches)
match_tot.columns = [i.lower() for i in match_tot.columns]
match_tot.to_csv("matches.csv") #panda method that writes the dataframe to a csv file